<a href="https://colab.research.google.com/github/ndvp39/crawler/blob/main/HW2/rottentomatoes_crawler.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

max 14 pages per genre (can be bigger)

crawler:

crawling the rotten tomatoes web and getting pages

In [ ]:

!pip install pyppeteer


In [ ]:
!pip install nest_asyncio


In [ ]:
!apt-get update
!apt-get install -y wget curl unzip xvfb
!wget https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb
!dpkg -i google-chrome-stable_current_amd64.deb
!apt-get -f install -y


Hit:1 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:2 https://dl.google.com/linux/chrome/deb stable InRelease
Hit:3 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Ign:6 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:7 https://r2u.stat.illinois.edu/ubuntu jammy Release
Hit:8 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:9 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:10 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:12 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not

In [ ]:
import subprocess
import os

# Start Xvfb
subprocess.Popen(['Xvfb', ':99', '-screen', '0', '1024x768x24'])
os.environ['DISPLAY'] = ':99'


In [ ]:
import asyncio
from pyppeteer import launch

In [ ]:
import requests
import json
from bs4 import BeautifulSoup
import nest_asyncio
web_url = 'https://www.rottentomatoes.com'

# Apply nest_asyncio to allow nested event loops in Jupyter/Colab
nest_asyncio.apply()

# loading more pages in a genre
async def click_load_more(page):
    try:
        # Wait until the "Load More" button is present and clickable
        await page.waitForSelector('button[data-qa="dlp-load-more-button"]', timeout=10000)

        # Get the previous count of movie items
        prev_movies_count = await page.evaluate('document.querySelectorAll("div[data-qa=\'discovery-media-list-item\']").length')

        # Click the "Load More" button
        await page.click('button[data-qa="dlp-load-more-button"]')

        # Wait for new content to be loaded by checking if the count of movie items has increased
        await page.waitForFunction(
            f'document.querySelectorAll("div[data-qa=\'discovery-media-list-item\']").length > {prev_movies_count}',
            timeout=60000
        )

        # Get the updated page source
        updated_html = await page.content()

        return updated_html

    except Exception as e:
        print(f"Error clicking 'Load More' button: {e}")
        return None





def extract_movie_details_links(genre_page):
    """Extract movie details links from the genre page."""
    movie_links = set()

    # Find all 'a' tags with the specific class that contains the movie links
    # the links to the movie info is in one of the 2 data-qa
    anchor_tags_1 = genre_page.find_all('a', {'data-qa': 'discovery-media-list-item'})
    anchor_tags_2 = genre_page.find_all('a', {'data-qa': 'discovery-media-list-item-caption'})

    for tag in anchor_tags_1 + anchor_tags_2:
        if 'href' in tag.attrs:
            movie_links.add(tag['href']) # added to a set for no repeating

    return list(movie_links)


def get_movie_name(movie_details_page):
    """Extract movie name from the movie details page."""
    return movie_details_page.find('h1').text.strip()

def get_movie_details(movie_details_link, processed_movies):
    """Fetch and parse movie details from a movie details link."""
    response = requests.get(web_url + movie_details_link)
    html_content = response.text
    movie_details_page = BeautifulSoup(html_content, 'html.parser')

    movie_name = get_movie_name(movie_details_page)
    move_release_date = get_movie_release_data(movie_details_page) # used if there is a movie with the same name but different date
    movie_key = movie_name + '_' + move_release_date
    if movie_key in processed_movies:
      # print('1. ' + movie_key)
      return None
    processed_movies.add(movie_key)
    # print('2. ' + movie_key)

    director_name = get_director_name(movie_details_page)
    runtime = get_runtime(movie_details_page)
    audience_score, critics_score = parse_scores_and_votes(movie_details_page)
    box_office = get_boxoffice(movie_details_page)

    movie_details = {
        'Movie Name': movie_name,
        'Release Date': move_release_date,
        'Director Name': director_name,
        'Runtime': runtime,
        'Audience Score': audience_score,
        'Critics Score': critics_score,
        'Box Office': box_office,
        'Movie URL': web_url + movie_details_link
    }

    return movie_details

# going to movies at home tab in the web
def get_movies_at_home_link(url):
    html_content = fetch_html_content(url)

    if html_content:
        soup = BeautifulSoup(html_content, 'html.parser')
        # Find the <a> tag inside the <p> tag with class="h4" and data-qa="dvd-streaming-main-link"
        a_tag = soup.find('p', class_='h4', attrs={'data-qa': 'dvd-streaming-main-link'}).find('a')

        if a_tag:
            return a_tag['href']
        else:
            print("Movies at home link not found.")
            return None
    else:
        return None

def get_movie_release_data(movie_details_page):
  """Extract release date from the movie details page."""
  category_wraps = movie_details_page.find_all('div', class_='category-wrap')
  for category in category_wraps:
      key_element = category.find('dt', class_='key')
      if key_element and key_element.find('rt-text', class_='key').text.strip() == 'Release Date (Streaming)':
          release_data_element = category.find('dd').find('rt-text')
          if release_data_element:
              return release_data_element.text.strip()
  return ''


# get all genres links
def get_genre_links(url):
    html_content = fetch_html_content(url)
    if not html_content:
        return []

    soup = BeautifulSoup(html_content, 'html.parser')
    checkbox_group = soup.find('select-checkbox-group', {'name': 'genres'})

    genre_links = []
    if checkbox_group:
        checkboxes = checkbox_group.find_all('select-checkbox')
        for checkbox in checkboxes:
            genre_link = checkbox.get('value', '')
            genre_links.append(genre_link)

    return genre_links

def get_director_name(movie_details_page):
    """Extract director name from the movie details page."""
    person_items = movie_details_page.select('a[data-qa="person-item"]')
    for item in person_items:
        role = item.find('p', {'class': 'role'}).text.strip()
        if 'Director' in role:
            return item.find('p', {'class': 'name'}).text.strip()
    return None

def get_runtime(movie_details_page):
    """Extract runtime from the movie details page."""
    category_wraps = movie_details_page.find_all('div', class_='category-wrap')
    for category in category_wraps:
        key_element = category.find('dt', class_='key')
        if key_element and key_element.find('rt-text', class_='key').text.strip() == 'Runtime':
            runtime_element = category.find('dd').find('rt-text')
            if runtime_element:
                return runtime_element.text.strip()
    return None

def parse_scores_and_votes(movie_details_page):
    """Parse audience and critics scores and votes from the movie details page."""
    audience_score = None
    critics_score = None

    script_tag = movie_details_page.find('script', {'data-json': 'reviewsData'})
    if script_tag:
        reviews_data = json.loads(script_tag.string)
        if 'audienceScore' in reviews_data:
            audience_score = reviews_data['audienceScore']['scorePercent']
        if 'criticsScore' in reviews_data:
            critics_score = reviews_data['criticsScore']['scorePercent']

    return audience_score, critics_score

def get_boxoffice(movie_details_page):
    """Extract boxoffice from the movie details page."""
    category_wraps = movie_details_page.find_all('div', class_='category-wrap')
    for category in category_wraps:
        key_element = category.find('dt', class_='key')
        if key_element and key_element.find('rt-text', class_='key').text.strip() == 'Box Office (Gross USA)':
            boxoffice_element = category.find('dd').find('rt-text')
            if boxoffice_element:
                return boxoffice_element.text.strip()
    return None


def fetch_html_content(url):
    """Fetch HTML content from a given URL."""
    try:
        response = requests.get(url)
        response.raise_for_status()  # Raise exception for bad responses
        return response.text
    except requests.exceptions.RequestException as e:
        print(f"Error fetching {url}: {e}")
        return None

async def main():
    # Launching the browser in headless mode with specific arguments for sandboxing
    browser = await launch(headless=True, args=['--no-sandbox', '--disable-setuid-sandbox'])
    try:
        # Creating a new page/tab in the browser
        page = await browser.newPage()

        # Getting the link for the "Movies at Home" section
        url_movies_at_home = get_movies_at_home_link(web_url)
        if not url_movies_at_home:
            print("Movies at Home link not found.")
            return

        # Constructing the main URL using the base URL and the "Movies at Home" link
        main_url = web_url + url_movies_at_home

        # Retrieving all the genre links from the main URL
        genre_links = get_genre_links(main_url)

        # Initializing a dictionary to store movie data and a counter for total movies fetched
        db = {}
        total_movies_fetched = 0

        # Iterating through each genre link
        for c_link in genre_links:
            curr_link = main_url + '/genres:' + c_link
            genre = []
            print(f"Processing genre link: {curr_link}")

            # Navigating to the current genre link and waiting for the network to be idle
            await page.goto(curr_link, {'waitUntil': 'networkidle2', 'timeout': 60000})

            # Getting the page content after it has loaded
            last_updated_html = await page.content()
            load_more_cnt = 1

            # Looping to click the "Load More" button to load additional movies
            while True:
                if load_more_cnt == 14:
                    break
                print(f'page {load_more_cnt}')

                # Clicking the "Load More" button and updating the HTML content
                updated_html = await click_load_more(page)
                if updated_html:
                    load_more_cnt += 1
                    last_updated_html = updated_html
                else:
                    break

            # Parsing the final HTML content with BeautifulSoup
            genre_page = BeautifulSoup(last_updated_html, 'html.parser')
            print(f"Rendering genre {c_link}")

            # Set to keep track of processed movies and avoid duplicates
            processed_movies = set()

            # Extracting movie details links from the genre page
            movie_details_links = extract_movie_details_links(genre_page)

            # Iterating through each movie details link
            for movie_details_link in movie_details_links:
                # Getting movie details and adding them to the genre list
                movie_details = get_movie_details(movie_details_link, processed_movies)
                if movie_details:
                    genre.append(movie_details)
            # Adding the genre movies to the database dictionary
            db[c_link] = genre
            total_movies_fetched += len(genre)

        db_json = json.dumps(db, ensure_ascii=False, indent=2)
        print(db_json)
        print(f"Total movies fetched: {total_movies_fetched}")

    finally:
        await browser.close() # Closing the browser

# Run the main function
await main()

saving the data in db.json file


In [ ]:

from google.colab import drive
import json
import os
print(f"Total movies fetched: {total_movies_fetched}")
# Mount Google Drive
drive.mount('/content/drive')

# Convert your data to JSON format
db_json = json.dumps(db, ensure_ascii=False, indent=2)

# Define the path to the "crawler" folder in Google Drive
destination_folder = '/content/drive/My Drive/crawler'
destination_path = os.path.join(destination_folder, 'db.json')

# Create the "crawler" folder if it doesn't exist
os.makedirs(destination_folder, exist_ok=True)

# Save the JSON data to a file in the specified folder
with open(destination_path, 'w', encoding='utf-8') as f:
    f.write(db_json)

# Print the JSON content
print(db_json)

# Verify if the file has been saved correctly
print(f"File saved to: {destination_path}")


Streaming output truncated to the last 5000 lines.
      "Audience Score": "96%",
      "Critics Score": "98%",
      "Box Office": null,
      "Movie URL": "https://www.rottentomatoes.com/m/das_boot"
    },
    {
      "Movie Name": "Night and Fog",
      "Release Date": "Mar 1, 2017",
      "Director Name": "Alain Resnais",
      "Runtime": "32m",
      "Audience Score": "95%",
      "Critics Score": "100%",
      "Box Office": null,
      "Movie URL": "https://www.rottentomatoes.com/m/night_and_fog"
    },
    {
      "Movie Name": "The Last Full Measure",
      "Release Date": "Apr 7, 2020",
      "Director Name": "Todd Robinson",
      "Runtime": "1h 56m",
      "Audience Score": "91%",
      "Critics Score": "61%",
      "Box Office": "$2.9M",
      "Movie URL": "https://www.rottentomatoes.com/m/the_last_full_measure"
    },
    {
      "Movie Name": "Hondros",
      "Release Date": "Mar 6, 2018",
      "Director Name": "Greg Campbell",
      "Runtime": "1h 26m",
      "Audience 

Start here if you have database - getting the data base and find the answers to the queries

In [ ]:
def format_result_as_json(sorted_directors):
    return json.dumps(sorted_directors, indent=2)

In [ ]:
def parse_runtime(runtime_str):
    """
    Converts a runtime string in the format 'Xh Ym' into total minutes.

    Args:
    runtime_str (str): Runtime string to parse.

    Returns:
    int: Total runtime in minutes or None if runtime_str is None.
    """
    if runtime_str is None:
        return None
    time_parts = runtime_str.split()
    total_minutes = 0
    for part in time_parts:
        if 'h' in part:
            total_minutes += int(part.replace('h', '')) * 60
        elif 'm' in part:
            total_minutes += int(part.replace('m', ''))
    return total_minutes

def average_and_largest_runtime_by_genre(data):
    """
    Calculates the average and the longest runtime for each genre.

    Args:
    data (dict): Dictionary containing genres as keys and list of movies as values.

    Returns:
    dict: Dictionary with genre stats including average runtime and the longest movie.
    """
    genre_stats = {}
    longest_movie = None
    for genre, movies in data.items():
        total_runtime = 0
        count = 0
        largest_runtime = 0
        for movie in movies:
            runtime_str = movie.get('Runtime')
            runtime = parse_runtime(runtime_str)
            if runtime is not None:
                total_runtime += runtime
                count += 1
                if runtime > largest_runtime:
                    largest_runtime = runtime
                    longest_movie = movie
        if count > 0:
            average_runtime = total_runtime / count
            genre_stats[genre] = {
                "Highest_Movie": longest_movie,
                "average_runtime": average_runtime
            }
        else:
            genre_stats[genre] = {
                "Highest_Movie": None,
                "average_runtime": None
            }
    return genre_stats

def sort_genre_runtimes_by_average(genre_stats):
    """
    Sorts genres by their average runtime in descending order.

    Args:
    genre_stats (dict): Dictionary with genre stats including average runtime and the longest movie.

    Returns:
    dict: Sorted dictionary with genres by average runtime.
    """
    # Filter out genres where average_runtime is None
    genre_stats_filtered = {genre: stats for genre, stats in genre_stats.items() if stats["average_runtime"] is not None}

    # Sort by average_runtime in descending order
    sorted_genre_stats = dict(sorted(genre_stats_filtered.items(), key=lambda item: item[1]["average_runtime"], reverse=True))

    return sorted_genre_stats

def get_sorted_genre_runtimes(db):
    """
    Retrieves sorted genre runtimes from the database.

    Args:
    db (dict): Database dictionary with genres and their movies.

    Returns:
    dict: Sorted dictionary with genres by average runtime.
    """
    genre_runtimes = average_and_largest_runtime_by_genre(db)
    sorted_genre_runtimes = sort_genre_runtimes_by_average(genre_runtimes)
    return sorted_genre_runtimes


In [ ]:
def parse_box_office(box_office):
    """
    Parses a box office string and converts it into a float value representing the total amount in dollars.

    Args:
    box_office (str): Box office string to parse.

    Returns:
    float: Total box office amount in dollars or 0 if the string is invalid or not provided.
    """
    if box_office:
        # Remove dollar sign and commas
        box_office = box_office.replace('$', '').replace(',', '')
        # Convert box office string to float value
        if 'M' in box_office:
            return float(box_office.replace('M', '')) * 1_000_000
        elif 'K' in box_office:
            return float(box_office.replace('K', '')) * 1_000
        elif 'B' in box_office:
            return float(box_office.replace('B', '')) * 1_000_000_000
    return 0

def find_highest_box_office_movie(data):
    """
    Finds the movie with the highest box office value for each genre.

    Args:
    data (dict): Dictionary containing genres as keys and list of movies as values.

    Returns:
    dict: Dictionary with the highest box office movie for each genre.
    """
    highest_box_office_movies = {}

    # Iterate through each genre and its movies
    for genre, movies in data.items():
        highest_box_office = 0
        highest_box_office_movie = None

        # Iterate through each movie in the genre
        for movie in movies:
            # Parse the box office value
            box_office_value = parse_box_office(movie["Box Office"])

            # Update the highest box office movie if a higher value is found
            if box_office_value > highest_box_office:
                highest_box_office = box_office_value
                highest_box_office_movie = movie

        # Store the highest box office movie for the genre
        if highest_box_office_movie:
            highest_box_office_movies[genre] = highest_box_office_movie

    return highest_box_office_movies



In [ ]:
def parse_score(score):
    """
    Parses a score string and converts it to an integer percentage.

    Args:
    score (str): Score string to parse.

    Returns:
    int: Parsed score as an integer percentage or 0 if parsing fails.
    """
    try:
        return int(score.strip('%'))
    except (AttributeError, TypeError):
        return 0

def aggregate_director_data(database):
    """
    Aggregates data for each director from the given database.

    Args:
    database (dict): Dictionary containing genres as keys and list of movies as values.

    Returns:
    dict: Aggregated data for each director, including total scores and movie count.
    """
    directors_data = {}

    for genre, movies in database.items():
        for movie in movies:
            director = movie["Director Name"]
            audience_score = parse_score(movie["Audience Score"])
            critics_score = parse_score(movie["Critics Score"])

            if director not in directors_data:
                directors_data[director] = {
                    "movies": [],
                    "total_audience_score": 0,
                    "total_critics_score": 0,
                    "count": 0
                }

            if movie not in directors_data[director]["movies"]:
                directors_data[director]["movies"].append(movie)
                directors_data[director]["total_audience_score"] += audience_score
                directors_data[director]["total_critics_score"] += critics_score
                directors_data[director]["count"] += 1

    return directors_data

def calculate_averages_and_combined_scores(directors_data):
    """
    Calculates average scores and a combined score for each director.

    Args:
    directors_data (dict): Dictionary containing aggregated data for each director.

    Returns:
    dict: Calculated averages and combined scores for each director.
    """
    result = {}

    for director, data in directors_data.items():
        if data["count"] > 0:
            # Calculate average scores
            avg_audience_score = data["total_audience_score"] / data["count"]
            avg_critics_score = data["total_critics_score"] / data["count"]

            # Normalize the combined score using the number of movies
            normalization_factor = data["count"] / (data["count"] + 1)
            combined_score = ((avg_audience_score + avg_critics_score) / 2) * normalization_factor
        else:
            avg_audience_score = 0
            avg_critics_score = 0
            combined_score = 0

        result[director] = {
            "Movies": data["movies"],
            "Audience Score Average": f"{avg_audience_score:.2f}%",
            "Critics Score Average": f"{avg_critics_score:.2f}%",
            "Combined Score": combined_score
        }

    return result

def sort_directors_by_score(directors):
    """
    Sorts directors by their combined score in descending order.

    Args:
    directors (dict): Dictionary containing directors and their calculated scores.

    Returns:
    list: Sorted list of directors by combined score.
    """
    return sorted(directors.items(), key=lambda x: x[1]["Combined Score"], reverse=True)

def get_directors_data(database):
    """
    Retrieves sorted director data based on scores from the database.

    Args:
    database (dict): Database dictionary with genres and their movies.

    Returns:
    list: Sorted list of directors by combined score.
    """
    directors_data = aggregate_director_data(database)
    directors_with_scores = calculate_averages_and_combined_scores(directors_data)
    sorted_directors = sort_directors_by_score(directors_with_scores)
    return sorted_directors


#print(get_directors_data(data))

getting the queries

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import json
import os

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


get the data from the db.json database file, and getting answers for the queries

In [ ]:
# Define the path to the JSON file in Google Drive
file_path = '/content/drive/My Drive/crawler/db.json'

# Read the JSON file
with open(file_path, 'r', encoding='utf-8') as f:
    data = json.load(f)

sorted_genre_runtimes = get_sorted_genre_runtimes(data)
highest_per_genre = find_highest_box_office_movie(data)
best_directors = get_directors_data(data)

#print(format_result_as_json(sorted_genre_runtimes)) # ג'אנרים עם ממוצע זמן סרט גדול ביותר ובכל ג'אנר, נותן גם את הסרט עם הזמן ריצה הגדול ביותר בתוך כל ז'אנר - ממוין
#print('-----------------------------------------------------------------------')
#print(format_result_as_json(highest_per_genre)) # סרטים הכי מרוויחים בכל ז'אנר
#print('-----------------------------------------------------------------------')
#print(format_result_as_json(best_directors)) # במאים מוצלחים ביותר - ממוין. נותן גם את הסרטים שיש לכל במאי

FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/My Drive/crawler/db.json'

saving the pages of the queries

In [ ]:
pages_from_boxoffice_query = [] # set()
for genre, items in highest_per_genre.items():
  pages_from_boxoffice_query.append(items['Movie URL']) # add

print(pages_from_boxoffice_query)
print(len(pages_from_boxoffice_query))

['https://www.rottentomatoes.com/m/avengers_endgame', 'https://www.rottentomatoes.com/m/avengers_endgame', 'https://www.rottentomatoes.com/m/incredibles_2', 'https://www.rottentomatoes.com/m/pokemon_the_first_movie_mewtwo_vs_mew', 'https://www.rottentomatoes.com/m/american_sniper', 'https://www.rottentomatoes.com/m/spider_man_no_way_home', 'https://www.rottentomatoes.com/m/the_batman', 'https://www.rottentomatoes.com/m/taylor_swift_the_eras_tour', 'https://www.rottentomatoes.com/m/titanic', 'https://www.rottentomatoes.com/m/avengers_endgame', 'https://www.rottentomatoes.com/m/bohemian_rhapsody', 'https://www.rottentomatoes.com/m/titanic', 'https://www.rottentomatoes.com/m/home_alone', 'https://www.rottentomatoes.com/m/exorcist', 'https://www.rottentomatoes.com/m/incredibles_2', 'https://www.rottentomatoes.com/m/bohemian_rhapsody', 'https://www.rottentomatoes.com/m/beauty_and_the_beast_2017', 'https://www.rottentomatoes.com/m/jurassic_world', 'https://www.rottentomatoes.com/m/wild_life_

In [ ]:
pages_from_runtime_query = [] # set()
for genre, info in sorted_genre_runtimes.items():
    highest_movie = info.get('Highest_Movie')
    if highest_movie:
        movie_url = highest_movie.get('Movie URL')
        if movie_url:
            pages_from_runtime_query.append(movie_url) # add

print(pages_from_runtime_query)
print(len(pages_from_runtime_query))

['https://www.rottentomatoes.com/m/1046038-gettysburg', 'https://www.rottentomatoes.com/m/zack_snyders_justice_league', 'https://www.rottentomatoes.com/m/the_lord_of_the_rings_the_return_of_the_king', 'https://www.rottentomatoes.com/m/the_irishman', 'https://www.rottentomatoes.com/m/lawrence_of_arabia', 'https://www.rottentomatoes.com/m/1046038-gettysburg', 'https://www.rottentomatoes.com/m/the_lord_of_the_rings_the_return_of_the_king', 'https://www.rottentomatoes.com/m/the_irishman', 'https://www.rottentomatoes.com/m/avatar_the_way_of_water', 'https://www.rottentomatoes.com/m/beau_is_afraid', 'https://www.rottentomatoes.com/m/gone_with_the_wind', 'https://www.rottentomatoes.com/m/mohabbatein', 'https://www.rottentomatoes.com/m/babylon_2022', 'https://www.rottentomatoes.com/m/heavens_gate', 'https://www.rottentomatoes.com/m/george_harrison_living_in_the_material_world', 'https://www.rottentomatoes.com/m/woodlands_dark_and_days_bewitched_a_history_of_folk_horror', 'https://www.rottentom

In [ ]:
pages_from_directors_query = [] # set()
for director, details in best_directors[:100]: # best 100 directors-> best_directors[:100]
    movies = details["Movies"]
    for movie in movies:
      pages_from_directors_query.append(movie["Movie URL"]) # add

print(pages_from_directors_query)
print(len(pages_from_directors_query))

['https://www.rottentomatoes.com/m/ponyo', 'https://www.rottentomatoes.com/m/kikis_delivery_service', 'https://www.rottentomatoes.com/m/the_boy_and_the_heron', 'https://www.rottentomatoes.com/m/spirited_away', 'https://www.rottentomatoes.com/m/princess_mononoke', 'https://www.rottentomatoes.com/m/howls_moving_castle', 'https://www.rottentomatoes.com/m/the_wind_rises', 'https://www.rottentomatoes.com/m/castle_in_the_sky', 'https://www.rottentomatoes.com/m/lupin_iii_the_castle_of_cagliostro', 'https://www.rottentomatoes.com/m/porco_rosso', 'https://www.rottentomatoes.com/m/nausicaa_of_the_valley_of_the_wind', 'https://www.rottentomatoes.com/m/my_neighbor_totoro', 'https://www.rottentomatoes.com/m/seven_samurai_1956', 'https://www.rottentomatoes.com/m/high_and_low', 'https://www.rottentomatoes.com/m/ikiru', 'https://www.rottentomatoes.com/m/ran', 'https://www.rottentomatoes.com/m/kagemusha', 'https://www.rottentomatoes.com/m/once_upon_a_time_in_america', 'https://www.rottentomatoes.com/m/

save the results from the queries in excel files

In [ ]:
!pip install pandas openpyxl
import pandas as pd

In [ ]:
def json_to_excel(data, excel_file):
    # Prepare data for the combined sheet
    combined_data = []

    for genre, details in data.items():
        movie_details = details["Highest_Movie"]
        movie_details["Genre"] = genre
        movie_details["Average Runtime"] = details["average_runtime"]
        combined_data.append(movie_details)

    # Create a DataFrame for the combined data
    df_combined = pd.DataFrame(combined_data)

    # Write to an Excel file
    df_combined.to_excel(excel_file, sheet_name="Movies Data", index=False)

json_to_excel(sorted_genre_runtimes, "/content/drive/My Drive/crawler/avarage_runtime_per_genre_results.xlsx")

In [ ]:
def write_movie_statistics_to_excel(data, file_path):
    """
    Writes movie statistics data to an Excel file.

    Parameters:
    data (dict): The movie statistics data.
    file_path (str): The path to the output Excel file.
    """
    # Convert the nested dictionary to a DataFrame
    df = pd.DataFrame.from_dict(data, orient='index')

    # Save the DataFrame to an Excel file
    df.to_excel(file_path, index_label='Genre')
write_movie_statistics_to_excel(highest_per_genre, "/content/drive/My Drive/crawler/best_boxoffice_per_genre_results.xlsx")

In [ ]:


def save_directors_to_excel(best_directors, excel_file):
    """
    Save director and movie data to an Excel file, with statistics and movies combined.

    Parameters:
    - best_directors (list of tuples): List of tuples where each tuple contains:
        - director (str): Director's name.
        - details (dict): Dictionary containing movie details and director statistics.
    - excel_file (str): Path to the Excel file where the data will be saved.
    """
    # Create an empty DataFrame for storing all movie data
    df_movies = pd.DataFrame()

    # Loop through each director and their data
    for director, details in best_directors:
        # Extract movies and convert to DataFrame
        movies = details['Movies']
        df_director_movies = pd.DataFrame(movies)

        # Add a column for the director's name
        df_director_movies['Director Name'] = director

        # Append to the main DataFrame
        df_movies = pd.concat([df_movies, df_director_movies], ignore_index=True)

    # Add director statistics
    df_stats = pd.DataFrame([
        {"Director Name": director,
         "Audience Score Average": details.get('Audience Score Average'),
         "Critics Score Average": details.get('Critics Score Average'),
         "Combined Score": details.get('Combined Score')}
        for director, details in best_directors
    ])

    # Combine movie data with director statistics
    df_combined = df_movies.merge(df_stats, on='Director Name', how='left')

    # Remove any unwanted columns, like 'Average Runtime'
    # List of columns to include in the final DataFrame
    columns_to_include = [
        'Movie Name', 'Release Date', 'Director Name', 'Runtime',
        'Audience Score', 'Critics Score', 'Box Office', 'Movie URL',
        'Audience Score Average', 'Critics Score Average', 'Combined Score'
    ]

    # Ensure only the specified columns are included in the final DataFrame
    df_combined = df_combined[columns_to_include]

    # Save to Excel
    with pd.ExcelWriter(excel_file) as writer:
        df_combined.to_excel(writer, sheet_name='Combined Data', index=False)

    print(f"Excel file has been created and saved as '{excel_file}'")

save_directors_to_excel(best_directors, '/content/drive/My Drive/crawler/best_directors_results.xlsx')

Excel file has been created and saved as '/content/drive/My Drive/crawler/best_directors_results.xlsx'


queries_results =
[
  pages_from_boxoffice_query,
  pages_from_runtime_query,
  pages_from_directors_query
]

using the pages return from the queries, calculating the index, inverted index

In [ ]:

!apt-get update
!apt-get install -y wget curl unzip xvfb
!wget https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb
!dpkg -i google-chrome-stable_current_amd64.deb
!apt-get -f install -y


Hit:1 https://dl.google.com/linux/chrome/deb stable InRelease
Hit:2 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:4 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Ign:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:6 https://r2u.stat.illinois.edu/ubuntu jammy Release
Hit:7 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:8 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:9 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:10 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:12 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not

In [ ]:
!pip install pyppeteer
!pip install nest_asyncio

In [ ]:
import subprocess
import osd

# Start Xvfb
subprocess.Popen(['Xvfb', ':99', '-screen', '0', '1024x768x24'])
os.environ['DISPLAY'] = ':99'

In [ ]:
from operator import invert
from pyppeteer import launch
import asyncio
from collections import defaultdict
from bs4 import BeautifulSoup
from heapq import nlargest
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
import nest_asyncio

# Apply nest_asyncio to allow asyncio in Jupyter/Colab environments
nest_asyncio.apply()

# Download NLTK stopwords if not already downloaded
nltk.download('stopwords')
nltk.download('punkt')

# Initialize stop words and stemmer
stop_words_english = set(stopwords.words('english'))
stop_words_spanish = set(stopwords.words('spanish'))

# Combine stopwords from both languages
stop_words = stop_words_english.union(stop_words_spanish)
stemmer = PorterStemmer()

async def click_load_more(page, previous_count):
    try:
        print("Waiting for 'Load More' button to be present...")

        # Attempt to wait for both selectors
        button_selector = 'rt-button[data-loadmoremanager="btnLoadMore:click"]'
        alternative_selector = '#reviews > div.load-more-container > rt-button'

        # Wait for the button to appear and be clickable
        await page.waitForSelector(button_selector, timeout=20000)
        button = await page.querySelector(button_selector)

        if not button:
            print("Primary button selector not found. Trying alternative selector...")
            await page.waitForSelector(alternative_selector, timeout=20000)
            button = await page.querySelector(alternative_selector)

        if button:
            print("Button found. Clicking 'Load More' button...")
            await button.click()

            # Wait for new content to load (you might need to adjust this based on your page's behavior)
            print("Waiting for new reviews to appear...")
            if not await wait_for_new_reviews(page, previous_count):
              print("No new reviews loaded.")
              return  False
            return True
        else:
            print("No 'Load More' button found with both selectors.")
            return False

    except Exception as e:
        print(f"Error clicking 'Load More' button: {e}")
        return False


async def wait_for_new_reviews(page, previous_count):
    try:
        print("Waiting for new reviews to load...")
        await page.waitForFunction(
            f'document.querySelectorAll("p[data-qa=\'review-quote\']").length > {previous_count} || document.querySelectorAll("p[data-qa=\'review-text\']").length > {previous_count}',
            timeout=15000
        )
        return True
    except Exception as e:
        print(f"Error waiting for new reviews: {e}")
        return False


async def fetch_and_extract_text(url):
    browser = None
    try:
        print(f"Opening browser for URL: {url}")
        browser = await launch(headless=True, args=['--no-sandbox', '--disable-setuid-sandbox'])
        page = await browser.newPage()
        await page.goto(url, waitUntil='networkidle2')

        all_reviews_text = set()
        previous_count = 0
        reviews_counter = 0
        loadmore_times = 2 # 2 pages of reviews to load -> max 80 reviews for a movie, 2 pages for critics reviews and 2 pages for audience reviews
        for i in range(loadmore_times):
            print(f"Processing page, load more {i + 1}")

            # Get the count of reviews on the current page
            page_source = await page.content()
            soup = BeautifulSoup(page_source, 'html.parser')
            reviews = soup.find_all('p', {'data-qa': 'review-quote'})  # for critics reviews
            if not reviews:
                reviews = soup.find_all('p', {'data-qa': 'review-text'})  # for audience reviews
            previous_count = len(reviews)
            reviews_counter += len(reviews)
            print(f"Reviews count: {previous_count}")

            # Save the reviews found on the current page
            for review in reviews:
                review_text = review.get_text(separator=' ', strip=True)
                all_reviews_text.add(review_text)
            if i == loadmore_times - 1:
              break
            if not await click_load_more(page, previous_count):
                print("Failed to click 'Load More' or no more content to load.")
                break

        return '\n\n'.join(all_reviews_text).strip(), reviews_counter

    except Exception as e:
        print(f"Error in fetch_and_extract_text: {e}")
        return "", 0

    finally:
        if browser:
            await browser.close()


# Function to preprocess text (e.g., tokenize, remove stop words, and stem)
async def preprocess_text(text):
    print("Preprocessing text")
    # print(text)
    tokens = word_tokenize(text.lower())
    filtered_tokens = [stemmer.stem(word) for word in tokens if word.isalpha() and word not in stop_words]
    return filtered_tokens

# Function to get word frequency index
async def get_word_frequency_index(links):
    cnt = 0
    word_index = defaultdict(lambda: [0] * len(links))
    titles = []
    all_reviews_counter = 0

    for i, link in enumerate(links):
        cnt += 1
        titles.append(link)  # Store the link (title)
        for review_page_route in ['/reviews', '/reviews?type=user']:  # Critics reviews and audience reviews pages
            print(f'Indexing process, fetching reviews... current link: {link}{review_page_route} ({cnt}/{len(links)})')
            print(f'{all_reviews_counter} Reviews were counted till now')
            text, reviews_counter = await fetch_and_extract_text(link + review_page_route)
            if text:
                print(text)
                all_reviews_counter += reviews_counter
                words = await preprocess_text(text)

                print("Indexing - Counting words")
                for word in words:
                    # print(f"Indexing - Word: {word}")
                    word_index[word][i] += 1
                print(f'words: {len(word_index)}')
    return word_index, titles, all_reviews_counter

# Function to get inverted index
def get_inverted_index(word_frequency_index, titles, most_common_words):
    inverted_index = defaultdict(set)

    for word in word_frequency_index:
        if word in most_common_words:
            for i, count in enumerate(word_frequency_index[word]):
                if count > 0:
                    inverted_index[word].add(titles[i])

    # Limit to the first 20 pages where each word appears
    for word in inverted_index:
        if len(inverted_index[word]) > 20:
            inverted_index[word] = set(list(inverted_index[word])[:20])

    return inverted_index


async def main():
    # Example queries_pages_result should be defined or imported from your existing data
    queries_pages_result = set(pages_from_boxoffice_query) | set(pages_from_runtime_query) | set(pages_from_directors_query)  # to avoid duplicates
    print(len(queries_pages_result))

    index, index_docs, all_reviews_counter = await get_word_frequency_index(list(queries_pages_result))

    print("Index Docs:", index_docs)

    most_common_words = nlargest(15, index, key=index.get)
    print("Most Common Words:", most_common_words)

    inverted_index = get_inverted_index(index, index_docs, most_common_words)
    print("Inverted Index:", inverted_index)

    return index, index_docs, inverted_index, most_common_words

# Run the main function in a Jupyter or Colab environment
index, index_docs, inverted_index, most_common_words = await main()


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Streaming output truncated to the last 5000 lines.

The Shootist is one of the most unconventional westerns ever to feature the name of John Wayne. The genre legend was one never to break away from simplistic narrative tropes throughout the decades largely due to his conservative nature, though he did occasionally take on alternative approaches in cases such as The Searchers (1956) as a means of making slight adjustments to the changing times. In The Shootist, he takes on a role perfect for his age. Given that he was dealing with terminal cancer at the time he played the role, portraying an ageing gunman battling cancer is a role befitting enough to produce an effective swan song for the man's career.
The Shootist is arguably the most character-driven film of John Wayne's career. While viewers may be deterred by the minimization of shootouts and horse chases, the depth of the film is frequently enough to compensate. It admittedly means that the film progresses at a slower rate than mos

'\nqueries_results = [pages_from_boxoffice_query, pages_from_runtime_query, pages_from_directors_query]\n\nword_count_queries = []\nmost_common_words_queries = []\ninverted_index_queries = []\n\nfor links in queries_results:\n    # Get word frequency index\n    word_count = get_word_frequency_index(links)\n    word_count_queries.append(word_count);\n    print("Word Count:", word_count)\n\n    # Find the top 15 most common words\n    most_common_words = nlargest(15, word_count, key=word_count.get)\n    most_common_words_queries.append(most_common_words)\n    print("Most Common Words:", most_common_words)\n\n    # Get inverted index for common words\n    inverted_index = get_inverted_index(links, most_common_words)\n    inverted_index_queries.append(inverted_index)\n    print("Inverted Index:", inverted_index)\n\n    # Display inverted index for common words\n    for word, pages in inverted_index.items():\n        print(f"{word}: {list(pages)}")\n'

saving the index and inverted index from in excel file

In [ ]:
print(len(index))

38168


In [ ]:
!pip install pandas openpyxl


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
from collections import defaultdict
import os

# Function to save data to Excel in chunks
def save_to_excel_chunked(word_index, titles, chunk_size=1000, base_file_path='/content/drive/My Drive/crawler/index'):
    # Split the word_index into chunks
    num_chunks = (len(word_index) + chunk_size - 1) // chunk_size  # Calculate number of chunks
    print(f"Total number of chunks: {num_chunks}")

    for i in range(num_chunks):
        chunk_start = i * chunk_size
        chunk_end = min((i + 1) * chunk_size, len(word_index))

        # Extract chunk of word_index
        chunk_data = {key: word_index[key] for key in list(word_index.keys())[chunk_start:chunk_end]}
        df = pd.DataFrame(chunk_data).T
        df.columns = titles

        # Save each chunk to a separate Excel file
        file_path = f'{base_file_path}_part_{i + 1}.xlsx'
        df.to_excel(file_path, index_label='term/doc')
        print(f"Chunk {i + 1}/{num_chunks} saved to file '{file_path}'")

        # Delete dataframe to free up memory
        del df

    print(f"Index Excel files have been saved successfully to {base_file_path}_part_*.xlsx.")

# Save to Excel in chunks
save_to_excel_chunked(index, index_docs)


Total number of chunks: 39
Chunk 1/39 saved to file '/content/drive/My Drive/crawler/index_part_1.xlsx'
Chunk 2/39 saved to file '/content/drive/My Drive/crawler/index_part_2.xlsx'
Chunk 3/39 saved to file '/content/drive/My Drive/crawler/index_part_3.xlsx'
Chunk 4/39 saved to file '/content/drive/My Drive/crawler/index_part_4.xlsx'
Chunk 5/39 saved to file '/content/drive/My Drive/crawler/index_part_5.xlsx'
Chunk 6/39 saved to file '/content/drive/My Drive/crawler/index_part_6.xlsx'
Chunk 7/39 saved to file '/content/drive/My Drive/crawler/index_part_7.xlsx'
Chunk 8/39 saved to file '/content/drive/My Drive/crawler/index_part_8.xlsx'
Chunk 9/39 saved to file '/content/drive/My Drive/crawler/index_part_9.xlsx'
Chunk 10/39 saved to file '/content/drive/My Drive/crawler/index_part_10.xlsx'
Chunk 11/39 saved to file '/content/drive/My Drive/crawler/index_part_11.xlsx'
Chunk 12/39 saved to file '/content/drive/My Drive/crawler/index_part_12.xlsx'
Chunk 13/39 saved to file '/content/drive/M

In [ ]:
!pip install XlsxWriter


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 159.9/159.9 kB 4.1 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import os

# Function to combine multiple Excel files into one with multiple sheets
def combine_excel_files(base_file_path, num_chunks):
    # Create an ExcelWriter object to write to a single Excel file
    combined_file_path = f'{base_file_path}_combined.xlsx'
    with pd.ExcelWriter(combined_file_path, engine='xlsxwriter') as writer:
        for i in range(num_chunks):
            file_path = f'{base_file_path}_part_{i + 1}.xlsx'
            if os.path.exists(file_path):
                # Read the chunked Excel file
                df = pd.read_excel(file_path, index_col='term/doc')
                # Write the DataFrame to a new sheet in the combined Excel file
                df.to_excel(writer, sheet_name=f'Chunk_{i + 1}')
                print(f"Added chunk {i + 1} to the combined Excel file.")
            else:
                print(f"File {file_path} does not exist.")

    print(f"All chunks have been combined into '{combined_file_path}' successfully.")

# Example usage
num_chunks = 39  # Update this to the actual number of chunk files you have
base_file_path = '/content/drive/My Drive/crawler/index'
combine_excel_files(base_file_path, num_chunks)


Added chunk 1 to the combined Excel file.
Added chunk 2 to the combined Excel file.
Added chunk 3 to the combined Excel file.
Added chunk 4 to the combined Excel file.
Added chunk 5 to the combined Excel file.
Added chunk 6 to the combined Excel file.
Added chunk 7 to the combined Excel file.
Added chunk 8 to the combined Excel file.
Added chunk 9 to the combined Excel file.
Added chunk 11 to the combined Excel file.
Added chunk 12 to the combined Excel file.
Added chunk 13 to the combined Excel file.
Added chunk 14 to the combined Excel file.
Added chunk 15 to the combined Excel file.
Added chunk 16 to the combined Excel file.
Added chunk 17 to the combined Excel file.
Added chunk 18 to the combined Excel file.
Added chunk 19 to the combined Excel file.
Added chunk 20 to the combined Excel file.
Added chunk 21 to the combined Excel file.
Added chunk 22 to the combined Excel file.
Added chunk 23 to the combined Excel file.
Added chunk 24 to the combined Excel file.
Added chunk 25 to th

In [ ]:
# Function to save the inverted index to an Excel file
def save_inverted_index_to_excel(inverted_index, file_path='/content/drive/My Drive/crawler/inverted_index_common_15.xlsx'):
    """
    Save the inverted index to an Excel file.

    Parameters:
    - inverted_index: A dictionary where the key is a word and the value is a set of URLs.
    - file_path: The path where the Excel file will be saved.
    """
    # Convert inverted_index to a dictionary
    inverted_index_dict = dict(inverted_index)

    # Determine the maximum number of URLs for the current inverted index
    max_urls = max(len(urls) for urls in inverted_index_dict.values())

    # Initialize an empty list to hold the data
    data = []

    # Prepare data for the DataFrame
    for word, urls in inverted_index_dict.items():
        urls_list = list(urls)  # Convert set to list
        row = [word] + urls_list + [''] * (max_urls - len(urls_list))  # Fill the row
        data.append(row)  # Append the row to the data list

    # Create DataFrame
    columns = ['Word'] + [f'Doc_{i+1}' for i in range(max_urls)]
    df = pd.DataFrame(data, columns=columns)

    # Save DataFrame to an Excel file
    df.to_excel(file_path, index=False, engine='openpyxl')  # Save the file

    print(f"Inverted Index Excel file has been saved successfully to {file_path}.")


# Save the inverted index to the Excel file
save_inverted_index_to_excel(inverted_index)

Inverted Index Excel file has been saved successfully to /content/drive/My Drive/crawler/inverted_index_common_15.xlsx.


Python Gui

In [ ]:
!pip install pandas openpyxl ipywidgets


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
from google.colab import drive
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML
import os

# Mount Google Drive
drive.mount('/content/drive')

# Load queries from Excel files
def load_queries():
    queries = {}
    queries["Best Box Office per Genre"] = pd.read_excel("/content/drive/MyDrive/results/best_boxoffice_per_genre_results.xlsx", dtype=str, engine='openpyxl')
    queries["Best Directors"] = pd.read_excel("/content/drive/MyDrive/results/best_directors_results.xlsx", dtype=str, engine='openpyxl')
    queries["Average Runtime per Genre"] = pd.read_excel("/content/drive/MyDrive/results/avarage_runtime_per_genre_results.xlsx", dtype=str, engine='openpyxl')
    return queries

# Save user responses to a file
def save_user_responses(query, responses):
    os.makedirs("/content/drive/MyDrive/results/user_results", exist_ok=True)
    with open(f"/content/drive/MyDrive/results/user_results/{query}_responses.txt", "a") as f:
        for response in responses:
            f.write(str(response) + "\n")

# Load the queries
queries = load_queries()

# Create dropdown to select query
query_dropdown = widgets.Dropdown(
    options=list(queries.keys()),
    description='Select Query:',
    disabled=False,
    style={'description_width': 'initial'}
)

# Create output area for displaying query results
output = widgets.Output()

# Create a button to save responses
send_button = widgets.Button(description="Send", button_style='success')

# Event handler for the send button
def on_send_button_clicked(b):
    query = query_dropdown.value
    responses = [response.value for response in response_widgets]
    if len(responses) == 10:
        save_user_responses(query, responses)
        with output:
            clear_output()
            display(widgets.HTML("<span style='color: green; font-weight: bold;'>Responses saved successfully!</span>"))
    else:
        with output:
            clear_output()
            display(widgets.HTML("<span style='color: red; font-weight: bold;'>Please rate all 10 results before sending.</span>"))

send_button.on_click(on_send_button_clicked)

# Function to display query results
response_widgets = []

def display_query_results(change):
    query = query_dropdown.value
    results = queries[query]

    with output:
        clear_output()
        response_widgets.clear()

        for i, result in results.iterrows():


            result_html = "<b>Result {}</b><br>".format(i + 1) + "<br>".join([f"<b>{key}:</b> {value}" for key, value in result.items()])
            result_label = widgets.HTML(result_html, layout=widgets.Layout(margin='10px 0'))
            response = widgets.RadioButtons(
                options=['', 'Plus', 'Minus'],
                description=f'Rating {i+1}:',
                disabled=False,
                style={'description_width': 'initial'}
            )

            response_widgets.append(response)
            display(widgets.VBox([result_label, response]))

query_dropdown.observe(display_query_results, names='value')

# Display the interface
app_layout = widgets.VBox([
    widgets.HTML("<h1>Movie Queries</h1>"),
    query_dropdown,
    send_button,
    output
], layout=widgets.Layout(margin='20px 0'))

display(app_layout)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
